In [6]:
from face_alignment import FaceDetection
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import time
import random
import json
from PIL import Image
from arcface import ArcFace
import numpy as np
from glob import glob
import os
import cv2
from agegender import AgeGender


# HERE = os.path.dirname(os.path.abspath(__file__))
HERE = os.getcwd()

ag = AgeGender()
af = ArcFace()
fd = FaceDetection(face_detector='blazeface')

with open('/home/tk/datasets/MELD/MELD.Raw/train/small_dataset.json', 'r') as stream:
    small_dataset = json.load(stream)

vid_root_path = '/home/tk/datasets/MELD/MELD.Raw/train/train_splits/'

random_dia = random.choice(small_dataset['train'])
print(f"random dialogue : {random_dia}")
print()

utts = glob(vid_root_path + random_dia + '*.mp4')
utts.sort()
print(f"number of utterance videos is {len(utts)}")
print(utts)
print()

utt = random.choice(utts)
print(f"random utt: {utt}")
print()

cap = cv2.VideoCapture(utt)
frames = []
while True:
    success, frame = cap.read()
    if not success:
        break

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frames.append(frame)

print(f"number of frames in this utt is {len(frames)}")



batch_size = 64
num_iter = len(frames) // batch_size
remainders = len(frames) % batch_size
landmarks = []
bboxes = []
faces = []

for i in tqdm(range(num_iter)):
    batch = np.stack(frames[i*batch_size: (i+1)*batch_size])
    print(batch.shape)

    bboxes_ = fd.detect_faces(batch)
    landmarks_ = fd.detect_landmarks(batch, bboxes_)
    faces_ = fd.crop_and_align(batch, bboxes_, landmarks_)
    for bb, lm, fa in zip(bboxes_, landmarks_, faces_):
        landmarks.append(lm)
        bboxes.append(bb)
        faces.append(fa)

if remainders != 0:
    batch = np.stack(frames[num_iter*batch_size:])
    print(batch.shape)

    bboxes_ = fd.detect_faces(batch)
    landmarks_ = fd.detect_landmarks(batch, bboxes_)
    faces_ = fd.crop_and_align(batch, bboxes_, landmarks_)

    for bb, lm, fa in zip(bboxes_, landmarks_, faces_):
        landmarks.append(lm)
        bboxes.append(bb)
        faces.append(fa)

num_real_bboxes = [len(bb) for bb in bboxes]
num_real_landmarks = [len(lm) for lm in landmarks]
num_real_faces = [len(fc) for fc in faces]


assert num_real_bboxes == num_real_landmarks == num_real_faces

num_real = num_real_bboxes

bboxes = np.concatenate(bboxes)
landmarks = np.concatenate(landmarks)
faces = np.concatenate(faces)

print(bboxes.shape, landmarks.shape, faces.shape)

age, gender = ag.predict(faces)
print(age, gender)

embeddings = af.predict(images)
print(embeddings)


[*] load ckpt from /home/tk/repos/cltl-face-all/arcface/./pretrained_models/arc_res50/e_8_b_40000.ckpt
random dialogue : dia474

number of utterance videos is 19
['/home/tk/datasets/MELD/MELD.Raw/train/train_splits/dia474_utt0.mp4', '/home/tk/datasets/MELD/MELD.Raw/train/train_splits/dia474_utt1.mp4', '/home/tk/datasets/MELD/MELD.Raw/train/train_splits/dia474_utt10.mp4', '/home/tk/datasets/MELD/MELD.Raw/train/train_splits/dia474_utt11.mp4', '/home/tk/datasets/MELD/MELD.Raw/train/train_splits/dia474_utt12.mp4', '/home/tk/datasets/MELD/MELD.Raw/train/train_splits/dia474_utt13.mp4', '/home/tk/datasets/MELD/MELD.Raw/train/train_splits/dia474_utt14.mp4', '/home/tk/datasets/MELD/MELD.Raw/train/train_splits/dia474_utt15.mp4', '/home/tk/datasets/MELD/MELD.Raw/train/train_splits/dia474_utt16.mp4', '/home/tk/datasets/MELD/MELD.Raw/train/train_splits/dia474_utt17.mp4', '/home/tk/datasets/MELD/MELD.Raw/train/train_splits/dia474_utt18.mp4', '/home/tk/datasets/MELD/MELD.Raw/train/train_splits/dia474

NameError: name 'images' is not defined